In [1]:
%pip install requests

In [2]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [3]:
client_id = "a3257d3e8f294fac939a9c3dd128cb73"
client_secret = "45b4f34f2ba447799b81e2a4c98395cb"

In [4]:
class SpotifyAPI(object):
    # default variables
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    base_url = "https://api.spotify.com"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        # if client_id / client_secret are not set raise exception
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode()) # encode to byte then convert to b64
        return client_creds_b64.decode() # decode the b64 from bytes
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        # sends a request to Spotify API
        r = requests.post(token_url, data=token_data, headers=token_headers)
        # checks if valid request
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
        data = r.json() # our token received from Spotify
        access_token = data['access_token'] # token needed to request data from Spotify
        expires_in = data['expires_in'] # seconds until the token expires
        now = datetime.datetime.now() # datetime right now to create datetime of when token expires
        expires = now + datetime.timedelta(seconds=expires_in) # gives expiration a datetime stamp
        self.access_token = access_token
        self.access_token_expires = expires # sets access_token_expires = expires
        self.access_token_did_expire = expires < now # boolean, checks if access token expired
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type="artists", version="v1"):
        base_url = self.base_url
        endpoint = f"{base_url}/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist'):
        base_url = self.base_url
        endpoint = f"{base_url}/v1/search"
        headers = self.get_resource_header()
        
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [5]:
spotify = SpotifyAPI(client_id, client_secret)

In [6]:
spotify.search("Time", search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
       'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
       'id': '0TnOYISbd1XYRBk9myaseg',
       'name': 'Pitbull',
       'type': 'artist',
       'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
   

In [7]:
spotify.get_artist("0TnOYISbd1XYRBk9myaseg")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
 'followers': {'href': None, 'total': 9288869},
 'genres': ['dance pop', 'latin', 'miami hip hop', 'pop', 'pop rap'],
 'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
 'id': '0TnOYISbd1XYRBk9myaseg',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb2dc40ac263ef07c16a95af4e',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051742dc40ac263ef07c16a95af4e',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1782dc40ac263ef07c16a95af4e',
   'width': 160}],
 'name': 'Pitbull',
 'popularity': 84,
 'type': 'artist',
 'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}

In [8]:
spotify.get_album("41zMFsCjcGenYKVJYUXU2n")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
   'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
   'id': '1hCkSJcXREhrodeIHQdav8',
   'name': 'Ramin Djawadi',
   'type': 'artist',
   'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',
  'JM',
  'JO',
  'J